## 1. Setup

### 1.1. Import packages

In [427]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [428]:
# Import the necessary packages
import os
import random
# import copy
from statistics import mean
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
from sklearn.model_selection import StratifiedGroupKFold
from tqdm import tqdm

In [429]:
# PyTorch
import torch
from torch.utils.data import Dataset, Subset, DataLoader
from torch.profiler import profile, record_function, schedule, ProfilerActivity

import torchvision
# from torchvision.io import read_image
from torchvision.utils import draw_bounding_boxes
from torchvision.ops import box_convert
import torchvision.transforms as T
from torchvision.transforms.functional import to_pil_image

In [430]:
# Image Augmentation
import albumentations as A
# from albumentations.pytorch.transforms import ToTensorV2

In [431]:
print(torch.__version__)
print(torchvision.__version__)
print(A.__version__)

1.12.1+cu113
0.13.1+cu113
1.2.1


In [432]:
# Set reproducibility
SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
# torch.cuda.manual_seed(SEED)
# torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.benchmark = True

In [433]:
PROJECT_PATH = '/content/drive/MyDrive/ML_Projects/How_many_Sparrows'
DATA_PATH = os.path.join(PROJECT_PATH, 'data')
IMAGE_PATH, BBOX_DATA_PATH = [os.path.join(DATA_PATH, 'raw', data_path) for data_path in ['images', 'bboxes/bounding_boxes.csv']]
TRAIN_FILE_PATH, TEST_FILE_PATH = [os.path.join(DATA_PATH, 'prepared', csv_file) for csv_file in ['train.csv', 'test.csv']]

In [434]:
# Set the device to be used to run the model
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Set profile parameters
PROFILE_SCHEDULE = schedule(skip_first=1, wait=2, warmup=1, active=1)
PROFACTIVITY = ProfilerActivity.CUDA if DEVICE == 'cuda' else ProfilerActivity.CPU

/usr/local/lib/python3.7/dist-packages/torch/profiler/profiler.py:231: UserWarning: Profiler won't be using warmup, this can skew profiler results
  warn("Profiler won't be using warmup, this can skew profiler results")


### 1.2. Helper functions

In [435]:
def get_image_transforms(aug_file_name=''):
    """Returns a Albumentation object and saves it to a JSON file 
    if aug_file_name is specified."""
    aug = A.ReplayCompose([
                           A.LongestMaxSize(1333, always_apply=True),  
                           A.SmallestMaxSize(800, always_apply=True),
                           A.RandomSizedBBoxSafeCrop(800, 800, p=0.3),
                           A.Rotate(180, p=0.4, border_mode=cv2.BORDER_CONSTANT, crop_border=True),
                           A.HorizontalFlip(p=0.6),
                           A.VerticalFlip(p=0.4),
                           A.ColorJitter(0.5, 0.5, 0.5, 0, p=0.7),
                           A.RandomRain(p=0.5),
                           A.OneOrOther(
                               A.Blur(10, p=0.7),
                               A.GaussianBlur((11, 21), p=0.3),
                               p=0.6
                               ),
                           ], 
                          A.BboxParams(format='coco', min_visibility=0.1, label_fields=['labels']),
                          p=0.8)
    
    if aug_file_name:
        A.save(aug, os.path.join(PROJECT_PATH, 'results', aug_file_name)) 

    return aug

In [436]:
def stratified_group_train_test_split(data, stratification_basis, groups):
    """Stratified splits data into training and test sets,
    taking into account groups, and returns the corresponding indices."""
    split = StratifiedGroupKFold(n_splits=2, shuffle=True, random_state=0)
    train_ids, test_ids = next(split.split(X=data, y=stratification_basis, groups=groups))
    return train_ids, test_ids

## 2. Object Detection Models

### 2.1. Pytorch

#### 2.1.1. Helper Functions

In [437]:
def collate_batch(batch):
    """Collate batches in Dataloader."""
    return tuple(zip(*batch))

In [438]:
def process_trace(prof):
    prof_out = prof.key_averages().table(sort_by=f'{DEVICE}_memory_usage', row_limit=10)
    print(prof_out)
    trace_path = os.path.join(PROJECT_PATH, 'tmp')

    if not os.path.exists(trace_path):
        os.mkdir(trace_path)

    prof.export_chrome_trace(os.path.join(trace_path, f'model_inference_trace_{prof.step_num}.json')) # chrome://tracing/

In [439]:
@torch.inference_mode()
def draw_bboxes_on_image(img, bboxes, scores=None):
    """Draws an image with bounding boxes from Tensors."""
    img_box = draw_bounding_boxes(img, boxes=bboxes, colors='orange', width=2)
    img = to_pil_image(img_box.detach())
    plt.figure(figsize=(8, 10))
    plt.imshow(img)
    plt.axis('off')
    ax = plt.gca()
    if scores is not None:
        for bb, sc in zip(bboxes, scores):
            x, y = bb.tolist()[:2]
            text_sc = f"{sc:0.2f}"
            ax.text(x, y, text_sc , fontsize=12, 
                    bbox=dict(facecolor='orange', alpha=0.5))
            
    plt.show()

In [440]:
def train_one_epoch(dataloader, model, optimizer, device='cpu'):
    """Passes a training step in one epoch."""
    accum_dict_losses = {}
    accum_model_loss = 0
    num_batches = len(dataloader)
    dataloader_progress = tqdm(dataloader, colour='blue')

    # Set a model to the training mode
    model.train()

    for images, targets in dataloader_progress:
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        # Сompute a model batch losses
        batch_dict_losses = model(images, targets)
        batch_model_loss = sum([loss for loss in batch_dict_losses.values()])

        # Accumulate statistics for computing the average losses per epoch
        accum_dict_losses.update({
            k: accum_dict_losses.get(k, 0) + v.item() for k, v in batch_dict_losses.items()
            })
        accum_model_loss += batch_model_loss.item()

        # Optimize the model parameters
        optimizer.zero_grad()
        batch_model_loss.backward()
        optimizer.step()

    # Compute the average losses
    epoch_dict_losses = {k: v / num_batches for k, v in accum_dict_losses.items()}
    epoch_model_loss = accum_model_loss / num_batches 
  
    return {'epoch_dict_losses': epoch_dict_losses, 
            'epoch_loss': epoch_model_loss}

In [441]:
@torch.inference_mode()
def eval_one_epoch(dataloader, model, device='cpu'):
    """Passes a inference evaluation step in one epoch."""
    accum_ratio_detected_obj = 0
    accum_model_score = 0
    results = []
    num_batches = len(dataloader)    
    dataloader_progress = tqdm(dataloader, colour='blue')
    
    # Set a model to the evaluation mode
    model.eval()

    for images, targets in dataloader_progress:
        images = [img.to(device) for img in images]

        # Get prediction results
        outputs = model(images)

        # Сompute a model batch statistics
        batch_num_gt_boxes = [len(t['boxes']) for t in targets]
        batch_num_pred_boxes = [len(out['boxes'][out['labels'] == 1]) for out in outputs]
        batch_model_score = 0

        if sum(batch_num_pred_boxes) != 0:
            batch_model_score = mean(
                [mean(out['scores'][out['labels'] == 1].tolist()[:gt+1]) 
                    for out, gt in zip(outputs, batch_num_gt_boxes)])
        
        # Accumulate statistics for computing the average values per epoch        
        accum_ratio_detected_obj += sum(batch_num_pred_boxes) / sum(batch_num_gt_boxes)
        accum_model_score += batch_model_score

        # Convert the images to RGB format and add to the outputs
        for img, out in zip(images, outputs):
            out.update(image_rgb=img.mul(255).to(torch.uint8))

        results += outputs
    
    # Compute the average score and the ratio of detected objects 
    epoch_ratio_detected_obj = accum_ratio_detected_obj / num_batches
    epoch_model_score = accum_model_score / num_batches

    return {'epoch_precent_detected_obj': epoch_ratio_detected_obj,
            'epoch_score': epoch_model_score, 
            'results': results}

In [442]:
def draw_image_with_model_outputs(model_results, draw_score_threshold=0, image_index=None):
    """Draws a random or given image with output boxes and scores greater than draw_score_threshold."""
    if image_index is None:
        image_index = random.randint(0, len(model_results['results'])-1)

    sample_model_res = model_results['results'][image_index]
    sample_draw = sample_model_res['scores'] > draw_score_threshold
    draw_bboxes_on_image(sample_model_res['image_rgb'], sample_model_res['boxes'][sample_draw], 
                         sample_model_res['scores'][sample_draw])   

In [443]:
def save_model_state(model_to_save, save_model_dir, best_stat_name):
    """Saves a model state dictionary."""
    fname = '_'.join(['best', best_stat_name, 'weights.pt'])
    filepath =  os.path.join(PROJECT_PATH, 'models', save_model_dir, fname)
    torch.save(model_to_save.state_dict(), filepath)

In [444]:
def run_train(train_dataloader, val_dataloader, model, epochs, use_lr_scheduler=True,
              model_name='best_model', process_trace_func=process_trace):
    """Run a training and evaluation loop of a model for a fixed number of epochs."""
    print("Device: ", DEVICE)
    
    best_epoch_loss = None
    model.to(DEVICE)
    
    params = [p for p in model.parameters() if p.requires_grad]
    # Construct an optimizer
    optimizer = torch.optim.SGD(params, lr=0.002, momentum=0.9)
    # Construct a learning rate scheduler
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3)

    if not use_lr_scheduler:
       optimizer = torch.optim.Adam(params)
       lr_scheduler = None  

    with profile(activities=[PROFACTIVITY], profile_memory=True, 
                 record_shapes=True, schedule=PROFILE_SCHEDULE, on_trace_ready=process_trace_func) as prof:
        with record_function('model_inference'):  

            for epoch in range(epochs):
                print(f"EPOCH [{epoch+1}/{epochs}]:")

                # Training step
                print("TRAIN:")
                train_res = train_one_epoch(train_dataloader, model, optimizer, DEVICE)
                print("  epoch loss: {0}:\n    {1}".format(train_res['epoch_loss'], 
                                                                train_res['epoch_dict_losses']))

                if lr_scheduler is not None:
                    lr_scheduler.step()        
                
                # Evaluation step
                print("EVAL:")
                eval_res = eval_one_epoch(val_dataloader, model, DEVICE)
                print("  about {0:%} of objects have been detected\n  epoch score: {1}".format(
                    eval_res['epoch_precent_detected_obj'], 
                    eval_res['epoch_score'])) 
                
                # Save a model with the minimum epoch loss value
                if  best_epoch_loss is None:
                    best_epoch_loss = train_res['epoch_loss']
                elif best_epoch_loss > train_res['epoch_loss']:
                    best_epoch_loss = train_res['epoch_loss']
                    save_model_state(model, model_name, 'loss')
                    print(f"Model is saved! --- The best loss: {best_epoch_loss}")
                
                print("-" * 60)
                prof.step()

    print("DONE!")
    return {'train_res': train_res,
            'eval_res': eval_res}

#### 2.1.2. Data Loading

In [445]:
class ImageBBoxDataset(Dataset):
    """A Dataset from csv to detect objects in images."""
    def __init__(self, csv_file_path, img_dir_path, bbox_path, 
                 img_transforms=None, bbox_transform=None):
        self.img_dir_path = img_dir_path
        self.img_df = pd.read_csv(csv_file_path)
        self.bbox_df = pd.read_csv(bbox_path)
        self.img_transforms = img_transforms
        self.bbox_transform = bbox_transform # (bbox_transform_fn, *bbox_transform_args) 

    def __len__(self):
        return self.img_df.shape[0]

    def __getitem__(self, idx):
        img_name = self.img_df.iloc[idx, 0]
        img_path = os.path.join(self.img_dir_path, img_name)
        image = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
        bboxes = self.bbox_df.loc[(self.bbox_df.image_name == img_name), 
                                 ['bbox_x', 'bbox_y', 'bbox_width', 'bbox_height']].values
        labels = torch.ones((bboxes.shape[0],), dtype=torch.int64) 

        if self.img_transforms:
            aug = self.img_transforms(image=image, bboxes=bboxes, labels=labels)
            image = aug['image']
            bboxes = aug['bboxes']
            
            # # Save an applied image transformation
            # if 'replay' in aug:
                # with open(os.path.join(PROJECT_PATH, 'results', 'replay-aug-transforms.json'), 'w') as f:
                #     f.write(str(aug['replay']))
                 
        image = T.ToTensor()(image)
        bboxes = torch.as_tensor(bboxes, dtype=torch.float)

        if self.bbox_transform:
            bboxes = self.bbox_transform[0](bboxes, *self.bbox_transform[1:])            
             
        target = {'boxes': bboxes,
                  'labels': labels}

        return image, target

In [446]:
dataset_params = {
    'img_dir_path': IMAGE_PATH,
    'bbox_path': BBOX_DATA_PATH, 
    'bbox_transform': (box_convert, 'xywh', 'xyxy')
}

train_dataset = ImageBBoxDataset(TRAIN_FILE_PATH, **dataset_params) # img_transforms=get_image_transforms('image-aug-tansforms')
test_dataset = ImageBBoxDataset(TEST_FILE_PATH, **dataset_params)

In [447]:
# Split data into training and validation sets
train_ids, val_ids = stratified_group_train_test_split(train_dataset.img_df['Name'], 
                                                       train_dataset.img_df['Number_HSparrows'], 
                                                       train_dataset.img_df['Author'])

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:880: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  UserWarning,


In [448]:
dl_params = {
    'batch_size': 2,
    'collate_fn': collate_batch
} # num_workers=4

train_dl = DataLoader(Subset(train_dataset, train_ids), shuffle=True, **dl_params)
val_dl = DataLoader(Subset(train_dataset, val_ids), **dl_params)
test_dl = DataLoader(test_dataset, **dl_params)

In [449]:
# # Uncomment to show a sample image with bounding boxes on it
# sample_img, sample_target = train_dataset[random.randint(0, len(train_dataset)-1)]
# draw_bboxes_on_image(sample_img.mul(255).to(torch.uint8), bboxes=sample_target['boxes'])

#### 2.1.2. Model Loading and Training

In [450]:
from torchvision.models.detection import (fasterrcnn_resnet50_fpn, fcos_resnet50_fpn,
                                          fasterrcnn_mobilenet_v3_large_fpn,
                                          fasterrcnn_mobilenet_v3_large_320_fpn)
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.fcos import FCOSClassificationHead

In [451]:
# Set general parameters for model training
NUM_CLASSES = 2 # 1 class (house sparrow) + background
EPOCHS = 1
DRAW_IMG_IDX = 255

##### 1. Faster R-CNN (ResNet)

In [452]:
# Load a Faster R-CNN model pre-trained on COCO
faster_rcnn_model = fasterrcnn_resnet50_fpn(weights='COCO_V1') 
            # rpn_nms_thresh=0.7, 
            # rpn_fg_iou_thresh=0.7,     --> training
            # rpn_bg_iou_thresh=0.3,     --> training
            # rpn_positive_fraction=0.5,     --> training
            # rpn_score_thresh=0.0,     --> inference
            # box_score_thresh=0.05,     --> inference
            # box_nms_thresh=0.5,     --> inference
            # box_detections_per_img=100,     
            # box_fg_iou_thresh=0.5,     --> training
            # box_bg_iou_thresh=0.5,     --> training
            # box_positive_fraction=0.25     --> training

            # rpn_anchor_generator=AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
            #                                      aspect_ratios=((0.5, 1.0, 2.0),))

# Look at the model's pre-trained head
print(faster_rcnn_model.roi_heads.box_predictor)

FastRCNNPredictor(
  (cls_score): Linear(in_features=1024, out_features=91, bias=True)
  (bbox_pred): Linear(in_features=1024, out_features=364, bias=True)
)


In [453]:
# Get number of input features for the predictor
in_features = faster_rcnn_model.roi_heads.box_predictor.cls_score.in_features

# Replace the pre-trained head with a new one
faster_rcnn_model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes=NUM_CLASSES)

# Look at the model's new head
print(faster_rcnn_model.roi_heads.box_predictor)

FastRCNNPredictor(
  (cls_score): Linear(in_features=1024, out_features=2, bias=True)
  (bbox_pred): Linear(in_features=1024, out_features=8, bias=True)
)


In [454]:
# # Run a model training cycle
# faster_rcnn_res = run_train(train_dl, val_dl, faster_rcnn_model, epochs=EPOCHS, 
#                             use_lr_scheduler=True, model_name='faster_rcnn')

In [455]:
# # Draw a image with output boxes and scores
# draw_image_with_model_outputs(faster_rcnn_res['eval_res'], draw_score_threshold=0.4, 
#                               image_index=DRAW_IMG_IDX)   

##### 2. Faster R-CNN (Mob)

In [456]:
# Load a Faster R-CNN model pre-trained on COCO
faster_rcnn_mob_model = fasterrcnn_mobilenet_v3_large_fpn(weights='COCO_V1')

# Look at the model's pre-trained head
print(faster_rcnn_mob_model.roi_heads.box_predictor)

FastRCNNPredictor(
  (cls_score): Linear(in_features=1024, out_features=91, bias=True)
  (bbox_pred): Linear(in_features=1024, out_features=364, bias=True)
)


In [457]:
# Get number of input features for the predictor
in_features_mob = faster_rcnn_mob_model.roi_heads.box_predictor.cls_score.in_features

# Replace the pre-trained head with a new one
faster_rcnn_mob_model.roi_heads.box_predictor = FastRCNNPredictor(in_features_mob, num_classes=NUM_CLASSES)

# Look at the model's new head
print(faster_rcnn_mob_model.roi_heads.box_predictor)

FastRCNNPredictor(
  (cls_score): Linear(in_features=1024, out_features=2, bias=True)
  (bbox_pred): Linear(in_features=1024, out_features=8, bias=True)
)


In [458]:
# # Run a model training cycle
# faster_rcnn_mob_res = run_train(train_dl, val_dl, faster_rcnn_mob_model, epochs=EPOCHS, 
#                                 use_lr_scheduler=True, model_name='faster_rcnn_mob')

In [459]:
# # Draw a image with output boxes and scores
# draw_image_with_model_outputs(faster_rcnn_mob_res['eval_res'], draw_score_threshold=0.4, 
#                               image_index=DRAW_IMG_IDX)   

##### 3. Faster R-CNN (Mob320)

In [460]:
# Load a Faster R-CNN model pre-trained on COCO
faster_rcnn_mob320_model = fasterrcnn_mobilenet_v3_large_320_fpn(weights='COCO_V1')

# Look at the model's pre-trained head
print(faster_rcnn_mob320_model.roi_heads.box_predictor)

FastRCNNPredictor(
  (cls_score): Linear(in_features=1024, out_features=91, bias=True)
  (bbox_pred): Linear(in_features=1024, out_features=364, bias=True)
)


In [461]:
# Get number of input features for the predictor
in_features_mob320 = faster_rcnn_mob320_model.roi_heads.box_predictor.cls_score.in_features

# Replace the pre-trained head with a new one
faster_rcnn_mob320_model.roi_heads.box_predictor = FastRCNNPredictor(in_features_mob320, num_classes=NUM_CLASSES)

# Look at the model's new head
print(faster_rcnn_mob320_model.roi_heads.box_predictor)

FastRCNNPredictor(
  (cls_score): Linear(in_features=1024, out_features=2, bias=True)
  (bbox_pred): Linear(in_features=1024, out_features=8, bias=True)
)


In [462]:
# # Run a model training cycle
# faster_rcnn_mob320_res = run_train(train_dl, val_dl, faster_rcnn_mob320_model, epochs=EPOCHS, 
#                                    use_lr_scheduler=True, model_name='faster_rcnn_mob320')

In [463]:
# # Draw a image with output boxes and scores
# draw_image_with_model_outputs(faster_rcnn_mob320_res['eval_res'], draw_score_threshold=0.4, 
#                               image_index=DRAW_IMG_IDX) 

##### 4. FCOS

In [464]:
# Load a FCOS model pre-trained on COCO
fcos_model = fcos_resnet50_fpn(weights='COCO_V1') 
                        # score_thresh=0.2, 
                        # nms_thresh=0.6, 
                        # detections_per_img=100,
                        # center_sampling_radius=1.5,

                        # anchor_generator = AnchorGenerator(sizes=((8,), (16,), (32,), (64,), (128,)),
                        #                                    aspect_ratios=((1.0,),))

# Look at the model's pre-trained classification head
print(fcos_model.head.classification_head) 

FCOSClassificationHead(
  (conv): Sequential(
    (0): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): GroupNorm(32, 256, eps=1e-05, affine=True)
    (2): ReLU()
    (3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): GroupNorm(32, 256, eps=1e-05, affine=True)
    (5): ReLU()
    (6): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): GroupNorm(32, 256, eps=1e-05, affine=True)
    (8): ReLU()
    (9): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): GroupNorm(32, 256, eps=1e-05, affine=True)
    (11): ReLU()
  )
  (cls_logits): Conv2d(256, 91, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)


In [465]:
# Change classification_head for prediction the given number of classes
fcos_model.head.classification_head.num_classes = NUM_CLASSES

# Get number of input channels for the classifier
in_channels = fcos_model.head.classification_head.cls_logits.in_channels

# Replace the pre-trained classification top layer with a new one
num_anchors = fcos_model.head.classification_head.num_anchors
fcos_model.head.classification_head.cls_logits = torch.nn.Conv2d(in_channels, NUM_CLASSES * num_anchors, 
                                                                 kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

# Look at the model's classification head
print(fcos_model.head.classification_head)

FCOSClassificationHead(
  (conv): Sequential(
    (0): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): GroupNorm(32, 256, eps=1e-05, affine=True)
    (2): ReLU()
    (3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): GroupNorm(32, 256, eps=1e-05, affine=True)
    (5): ReLU()
    (6): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): GroupNorm(32, 256, eps=1e-05, affine=True)
    (8): ReLU()
    (9): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): GroupNorm(32, 256, eps=1e-05, affine=True)
    (11): ReLU()
  )
  (cls_logits): Conv2d(256, 2, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)


In [466]:
# Run a model training cycle
# fcos_res = run_train(train_dl, val_dl, fcos_model, epochs=EPOCHS, 
#                      use_lr_scheduler=False, model_name='fcos')

In [467]:
# # Draw a image with output boxes and scores
# draw_image_with_model_outputs(fcos_res['eval_res'], draw_score_threshold=0.4, 
#                               image_index=DRAW_IMG_IDX) 

##### 1.2.2. Check train functions

In [468]:
check_dl = DataLoader(Subset(train_dataset, [0, 1, 255]), **dl_params)

In [469]:
# models: faster_rcnn_model, faster_rcnn_mob_model, faster_rcnn_mob320_model, fcos_model
check_res = run_train(check_dl, check_dl, faster_rcnn_mob_model, epochs=EPOCHS, use_lr_scheduler=True)

Device:  cpu
EPOCH [1/1]:
TRAIN:


100%|██████████| 2/2 [00:08<00:00,  4.01s/it]


  epoch loss: 1.9349839091300964:
    {'loss_classifier': 0.712329626083374, 'loss_box_reg': 1.0923421382904053, 'loss_objectness': 0.09301595701253973, 'loss_rpn_box_reg': 0.03729613497853279}
EVAL:


100%|██████████| 2/2 [00:03<00:00,  1.60s/it]


  about 1650.000000% of objects have been detected
  epoch score: 0.5561054535210133
------------------------------------------------------------
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                              aten::mul         8.62%     969.079ms         8.62%     969.724ms       1.142ms       2.86 Gb       2.86 Gb           849  
                                              aten::add         7.05%     792.431ms         7.10%     798.820ms       1.437ms       2.21 Gb       2.21 Gb           556  
    